# 7.4. Multiple Input and Multiple Output Channels

In [194]:
import torch
from d2l import torch as d2l

## 7.4.1. Multiple Input Channels

https://d2l.ai/_images/conv-multi-in.svg

In [195]:
multi_input = torch.tensor([[[1, 2, 3],
                             [4, 5, 6],
                             [7, 8, 9]],
                            [[0, 1, 2],
                             [3, 4, 5],
                             [6, 7, 8]]])

In [196]:
multi_kernel = torch.tensor([[[1, 2],
                              [3, 4]],
                             [[0, 1],
                              [2, 3]]])

In [197]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [198]:
# zip
a = torch.tensor([1, 2, 3, 4, 5, 6])
b = torch.tensor([2, 3, 4, 5, 6, 7])
c = torch.tensor([3, 4, 5, 6, 7, 8])
abc = zip(a, b, c)
for i in abc:
    print(i)

# zip：把可迭代的对象打包成一个总体的可迭代对象

(tensor(1), tensor(2), tensor(3))
(tensor(2), tensor(3), tensor(4))
(tensor(3), tensor(4), tensor(5))
(tensor(4), tensor(5), tensor(6))
(tensor(5), tensor(6), tensor(7))
(tensor(6), tensor(7), tensor(8))


In [199]:
corr2d_multi_in(multi_input, multi_kernel)

tensor([[ 56.,  72.],
        [104., 120.]])

## 7.4.2. Multiple Output Channels

In [200]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [201]:
d = torch.tensor([[1, 2],
                  [3, 4]])
d

tensor([[1, 2],
        [3, 4]])

In [202]:
d + 1

tensor([[2, 3],
        [4, 5]])

In [203]:
# stack
stack = torch.stack((d, d+1, d+2))
stack

tensor([[[1, 2],
         [3, 4]],

        [[2, 3],
         [4, 5]],

        [[3, 4],
         [5, 6]]])

In [204]:
stack.shape

torch.Size([3, 2, 2])

In [205]:
torch.cat((d, d+1, d+2))

tensor([[1, 2],
        [3, 4],
        [2, 3],
        [4, 5],
        [3, 4],
        [5, 6]])

In [206]:
torch.cat((d, d+1, d+2), 1)

tensor([[1, 2, 2, 3, 3, 4],
        [3, 4, 4, 5, 5, 6]])

In [207]:
torch.stack((d, d+1, d+2), 0)

tensor([[[1, 2],
         [3, 4]],

        [[2, 3],
         [4, 5]],

        [[3, 4],
         [5, 6]]])

In [208]:
torch.stack((d, d+1, d+2), 1)

tensor([[[1, 2],
         [2, 3],
         [3, 4]],

        [[3, 4],
         [4, 5],
         [5, 6]]])

In [209]:
torch.stack((d, d+1, d+2), 2)

tensor([[[1, 2, 3],
         [2, 3, 4]],

        [[3, 4, 5],
         [4, 5, 6]]])

## 7.4.3. 1X1 Convolutional Layer

https://d2l.ai/_images/conv-1x1.svg

The only computation of the 1X1 convolution occurs on the channel dimension.

1X1 的卷积核的唯一作用就是降维。

In [210]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape(c_i, h * w)
    K = K.reshape(c_o, c_i)
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [211]:
mock_x = torch.normal(0, 1, (3, 3, 3))
print(mock_x)
c_i, h, w = mock_x.shape
mock_x = mock_x.reshape(c_i, h * w)
print(mock_x)

mock_kernel = torch.normal(0, 1, (2, 3, 1, 1))
c_o = mock_kernel.shape[0]
print(mock_kernel)
mock_kernel = mock_kernel.reshape(c_o, c_i)
print(mock_kernel)

mock_y = torch.matmul(mock_kernel, mock_x)
print(mock_y.reshape(c_o, h, w))

tensor([[[ 0.2505,  0.0719,  0.4363],
         [ 0.9037,  0.5283, -1.2233],
         [-1.2680,  0.4430, -1.8630]],

        [[ 0.1447,  0.2217, -0.0504],
         [-0.5425, -1.3993,  1.4197],
         [ 0.4881,  0.3813, -0.1797]],

        [[ 1.8397,  0.5504, -0.9410],
         [-2.5451,  0.0730,  1.4625],
         [-0.0579,  0.7513,  0.1000]]])
tensor([[ 0.2505,  0.0719,  0.4363,  0.9037,  0.5283, -1.2233, -1.2680,  0.4430,
         -1.8630],
        [ 0.1447,  0.2217, -0.0504, -0.5425, -1.3993,  1.4197,  0.4881,  0.3813,
         -0.1797],
        [ 1.8397,  0.5504, -0.9410, -2.5451,  0.0730,  1.4625, -0.0579,  0.7513,
          0.1000]])
tensor([[[[ 0.3448]],

         [[-0.2995]],

         [[-0.8709]]],


        [[[-1.3604]],

         [[-0.2776]],

         [[-0.4611]]]])
tensor([[ 0.3448, -0.2995, -0.8709],
        [-1.3604, -0.2776, -0.4611]])
tensor([[[-1.5591, -0.5209,  0.9850],
         [ 2.6904,  0.5376, -2.1205],
         [-0.5329, -0.6158, -0.6756]],

        [[-1.2292, 

In [212]:
# torch.normal(means, std, out=None)
X = torch.normal(0, 1, (3, 3, 3))
X

tensor([[[ 0.6803, -2.7627, -0.3948],
         [-0.3583,  2.0889,  0.9373],
         [ 1.1903, -0.9876,  0.5363]],

        [[-0.7343,  1.0380, -0.5914],
         [-0.7646, -1.7979,  1.2848],
         [-1.4764,  1.6627, -0.0360]],

        [[-1.1171,  0.2886,  0.2051],
         [ 0.4885, -0.5717, -1.6586],
         [ 0.8954, -1.0307,  0.6446]]])

In [213]:
K = torch.normal(0, 1, (2, 3, 1, 1))
K

tensor([[[[ 2.1294]],

         [[ 1.1924]],

         [[ 0.5782]]],


        [[[-1.0393]],

         [[ 0.1147]],

         [[-0.4848]]]])

In [214]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y1

tensor([[[-0.0728, -4.4782, -1.4273],
         [-1.3924,  1.9736,  2.5689],
         [ 1.2918, -0.7164,  1.4718]],

        [[-0.2497,  2.8503,  0.2431],
         [ 0.0479, -2.0999, -0.0226],
         [-1.8404,  1.7168, -0.8740]]])

In [215]:
Y2 = corr2d_multi_in_out(X, K)
Y2

tensor([[[-0.0728, -4.4782, -1.4273],
         [-1.3924,  1.9736,  2.5689],
         [ 1.2918, -0.7164,  1.4718]],

        [[-0.2497,  2.8503,  0.2431],
         [ 0.0479, -2.0999, -0.0226],
         [-1.8404,  1.7168, -0.8740]]])

In [216]:
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6